<a href="https://colab.research.google.com/github/tusker4/Sesac_Saltlux_DeepLearning/blob/main/8_NLP_koBert%EA%B8%B0%EB%B0%98_%EB%8B%A4%EC%A4%91%EB%B6%84%EB%A5%98_TPU_%EC%A0%84%EC%9D%B4%ED%95%99%EC%8A%B5_%ED%8C%8C%EC%9D%B8%ED%8A%9C%EB%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TPU 체크
- 구글에서 만든 연산장비
  - GPU 빠른 것으로 나와있지만 , 건 by 건으로 생각하면된다.
  - 구글클라우드에서만 사용 가능
- 무료 코랩으로 사용시 제공,
  - 전체 GPU, TPU 사용시간 제한 존재
  - TPU 경우는 할당이 안될 수 있다
  

In [ ]:
import os

try:
  print( os.environ['COLAB_TPU_ADDR'] )
except Exception as e:
  print( 'TPU 지원하지 않음')

10.42.13.226:8470


# 개요

- 다중분류
- 데이터
  - https://github.com/kakaobrain/KorNLUDatasets
  - NLI
    - 두 문장 사이의 관계를 표현
    - Entailment, Contradicton, Neutral
  - STS
    - 두 문장 사이의 의미적 유사상 평가한 데이터
    - 0 ~ 5 (동일한 의미)
-


# 데이터 획득 및 전처리

In [ ]:
# 바이너리 데이터(파일링크)를 직접 다운로드 처리
import urllib.request as req

url_common = 'https://raw.githubusercontent.com/kakaobrain/kor-nlu-datasets/master/'

# (원격지경로, 로컬 경로)
req.urlretrieve( url_common + "KorNLI/multinli.train.ko.tsv", filename="multinli.train.ko.tsv")
req.urlretrieve( url_common + "KorNLI/snli_1.0_train.ko.tsv", filename="snli_1.0_train.ko.tsv")
req.urlretrieve( url_common + "KorNLI/xnli.dev.ko.tsv",       filename="xnli.dev.ko.tsv")
req.urlretrieve( url_common + "KorNLI/xnli.test.ko.tsv",      filename="xnli.test.ko.tsv")

req.urlretrieve( url_common + "KorSTS/sts-dev.tsv",           filename="sts-dev.tsv")
req.urlretrieve( url_common + "KorSTS/sts-test.tsv",          filename="sts-test.tsv")
req.urlretrieve( url_common + "KorSTS/sts-train.tsv",         filename="sts-train.tsv")

('sts-train.tsv', <http.client.HTTPMessage at 0x781aa0cec0a0>)

In [ ]:
import pandas as pd

In [ ]:
train_multinli = pd.read_csv('multinli.train.ko.tsv', sep='\t', quoting=3) # quoting=3 : 인용구(따음표) 무시
train_snli     = pd.read_csv('snli_1.0_train.ko.tsv', sep='\t', quoting=3)
val_xnli       = pd.read_csv('xnli.dev.ko.tsv', sep='\t', quoting=3)
test_xnli      = pd.read_csv('xnli.test.ko.tsv', sep='\t', quoting=3)

val_sts        = pd.read_csv('sts-dev.tsv', sep='\t', quoting=3)
test_sts       = pd.read_csv('sts-test.tsv', sep='\t', quoting=3)
train_sts      = pd.read_csv('sts-train.tsv', sep='\t', quoting=3)


In [ ]:
train_sts.head(5)

,genre,filename,year,id,score,sentence1,sentence2
0,main-captions,MSRvid,2012test,1,5.00,비행기가 이륙하고 있다.,비행기가 이륙하고 있다.
1,main-captions,MSRvid,2012test,4,3.80,한 남자가 큰 플루트를 연주하고 있다.,남자가 플루트를 연주하고 있다.
2,main-captions,MSRvid,2012test,5,3.80,한 남자가 피자에 치즈를 뿌려놓고 있다.,한 남자가 구운 피자에 치즈 조각을 뿌려놓고 있다.
3,main-captions,MSRvid,2012test,6,2.60,세 남자가 체스를 하고 있다.,두 남자가 체스를 하고 있다.
4,main-captions,MSRvid,2012test,9,4.25,한 남자가 첼로를 연주하고 있다.,자리에 앉은 남자가 첼로를 연주하고 있다.


In [ ]:
train_multinli.shape, train_snli.shape, val_xnli.shape, test_xnli.shape

((392702, 3), (550152, 3), (2490, 3), (5010, 3))

In [ ]:
val_sts.shape, test_sts.shape, train_sts.shape

((1500, 7), (1379, 7), (5749, 7))

In [ ]:
train_snli.head(1)

,sentence1,sentence2,gold_label
0,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,한 사람이 경쟁을 위해 말을 훈련시키고 있다.,neutral


In [ ]:
train_sts.head(1)

,genre,filename,year,id,score,sentence1,sentence2
0,main-captions,MSRvid,2012test,1,5.0,비행기가 이륙하고 있다.,비행기가 이륙하고 있다.


In [ ]:
# train_multinli, train_snli 병합
train_nli = train_multinli.append( train_snli )  # concat으로 대체
train_nli.shape

<ipython-input-19-e22c7aca8baa>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_nli = train_multinli.append( train_snli )  # concat으로 대체


(942854, 3)

In [ ]:
train_nli [ train_multinli.shape[0]-2 : train_multinli.shape[0]+2  ]
# 인덱스가 깨짐 => 인덱스 조정(자체적 진행)


In [ ]:
# 결측제거, 중복제거, 인덱스 리셋
def data_clean( df ):
  df = df.dropna()
  df = df.drop_duplicates()
  df = df.reset_index(drop=True)
  return df

In [ ]:
train_nli = data_clean(train_nli)
vali_xnli = data_clean(val_xnli)
text_xnli = data_clean(test_xnli)

train_nli.shape, val_xnli.shape, test_xnli.shape

((941814, 3), (2490, 3), (5010, 3))

In [ ]:
# 정답 3개 => 다중분류

# 토큰화 작업

In [ ]:
!pip install transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.6 MB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizer, TFBertModel

In [ ]:
repo = 'klue/bert-base'
tokenizer = BertTokenizer.from_pretrained( repo )

In [ ]:
# 샘플 데이터 준비
sent1 , sent2 = train_nli.sentence1.iloc[0], train_nli.sentence1.iloc[1]
sent1 , sent2

('개념적으로 크림 스키밍은 제품과 지리라는 두 가지 기본 차원을 가지고 있다.',
 '시즌 중에 알고 있는 거 알아? 네 레벨에서 다음 레벨로 잃어버리는 거야 브레이브스가 모팀을 떠올리기로 결정하면 브레이브스가 트리플 A에서 한 남자를 떠올리기로 결정하면 더블 A가 그를 대신하러 올라가고 A 한 명이 그를 대신하러 올라간다.')

In [ ]:
max_length = 128  # 최대길이 설정
# 패딩시 최대길이 적용 => 모든 문장은 128개 토큰으로 구성 (더 큰 문장이 존재할 경우 정보 손실 발생)
sample = tokenizer.encode_plus(sent1, sent2, max_length, pad_to_max_length=True)
'''
  input_ids   : 문장의 백터화 처리 = 스페셜토큰 + 문장 1 + 스페셜토큰 + 문장 2 + 스페셜토큰 + 패딩값
  token_type_ids : 문장 1=0, 문장 2=1, 패딩=0
  atention_mask : 1=문장, 0=패
'''
sample

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


{'input_ids': [2, 4453, 11187, 6024, 7374, 2633, 2073, 3808, 2145, 6959, 23548, 864, 3663, 4116, 4540, 2069, 3663, 2088, 1513, 2062, 18, 3, 4161, 1570, 2170, 22819, 1513, 2259, 568, 4860, 35, 752, 12217, 27135, 3729, 12217, 2200, 8980, 2059, 2259, 568, 2275, 26094, 19809, 2116, 1078, 2628, 2069, 8636, 2015, 2200, 3794, 5643, 26094, 19809, 2116, 14448, 37, 27135, 1891, 3997, 2138, 8636, 2015, 2200, 3794, 5643, 8881, 37, 2116, 636, 2138, 4217, 2205, 2030, 5624, 2088, 37, 1891, 1076, 2052, 636, 2138, 4217, 2205, 2030, 16172, 18, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

- 위의 샘플을 기반으로 전체 데이터를 위의 모양으로 처리
  - 처리후 저장
    - 배열의 압축저장
      - np.savez_compressed()

In [ ]:
from tqdm import tqdm # 진행율 표현

for s1, s2 in tqdm(zip( train_nli.sentence1, train_nli.sentence2 ), total=train_nli.shape[0]):
    print(s1, s2)
    break

  0%|          | 0/941814 [00:00<?, ?it/s]

개념적으로 크림 스키밍은 제품과 지리라는 두 가지 기본 차원을 가지고 있다. 제품과 지리학은 크림 스키밍을 작동시키는 것이다.


In [ ]:
import numpy as np
def sentence_to_encode( df ):
    '''
        리턴
        input_ids
        token_type_ids
        attention_mask
    '''
    input_ids       = list()
    token_type_ids  = list()
    attention_mask  = list()
    for s1, s2 in tqdm(zip( df.sentence1, df.sentence2 ), total=df.shape[0]):
        vec_dict = tokenizer.encode_plus(s1, s2, max_length=max_length, pad_to_max_length=True)
        # 필요한 정보를 성분별로 추출
        input_ids.append(     vec_dict['input_ids'])
        token_type_ids.append(vec_dict['token_type_ids'])
        attention_mask.append(vec_dict['attention_mask'])
        pass

    #pass
    # 배열 압축 저장을 위해서, 배열 변환
    return np.array(input_ids, dtype=int), \
           np.array(token_type_ids, dtype=int),  \
           np.array(attention_mask, dtype=int)


X_train = sentence_to_encode( train_nli )
X_val   = sentence_to_encode( val_xnli  )
X_test  = sentence_to_encode( test_xnli )

  0%|          | 82/941814 [00:00<39:04, 401.73it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
  0%|          | 3832/941814 [00:11<1:09:02, 226.43it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have be

In [ ]:
# 정답 -> 수치화(인코딩 처리, 레이블인코더)
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit( train_nli.gold_label )

# 변환처리
y_train = encoder.transform( train_nli.gold_label)
y_val   = encoder.transform( val_xnli.gold_label)
y_test  = encoder.transform( test_xnli.gold_label )

In [ ]:
# 배열 데이터 저장
with open('/content/drive/MyDrive/머신러닝(솔트룩스)/딥 러닝/딥러닝_NLP/categorial.npy', 'wb') as f:
    np.savez_compressed(f, X_train=X_train, X_val=X_val, X_test=X_test, \
                           y_train=y_train,  y_val=y_val, y_test=y_test
                        )

# 데이터 로드